<a href="https://colab.research.google.com/github/TSHP/DL4SC_Neural_ODEs/blob/main/ToyExampleTobia.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Loading in the MNIST DATASET

In [2]:
print("hello world!")

hello world!
